# EDGAR XBRL XML URL

Generate URLs to the XBRL XML files in the fileing directory in a parallel processing manner using Ray.

In [ ]:
# !pip install -q ray

In [1]:
from typing import (
    List,
    Dict,
    Iterable
)
import os
import sys
import random
import logging
import time
import re
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import ray

In [2]:
pd.set_option('display.float_format', lambda x: ('%f' % x).rstrip('0').rstrip('.'))
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.max_columns', None)  

logging.basicConfig(level=logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)

Logger = logging.getLogger(__name__)
Logger.addHandler(handler)

In [3]:
Logger.debug("test")

test


DEBUG:__main__:test


In [4]:
from sec_edgar_constant import (
    NUM_CPUS,
    FS_TYPE_10K,
    FS_TYPE_10Q,
    EDGAR_HTTP_HEADERS,
    DATA_DIR_LISTING,
    DATA_DIR_XBRL,
)
from sec_edgar_utility import (
    split,
    http_get_content,
)
from sec_edgar_download_xbrl_xml import (
    save_to_xml,
    worker,
)

In [5]:
def pd_print_full():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

# Data 

## EDGAR XBRL Directory Indices

In [31]:
import pathlib
years = [2010]
qtrs = [1,2,3,4]
for YEAR in years:
    for QTR in qtrs:
        print("-" * 80)
        print(f"Processing {YEAR}/{QTR}")
        try:
            listing = pd.read_csv(
                f"{DATA_DIR_LISTING}/{YEAR}QTR{QTR}_XBRL.gz", 
                sep="|",
                usecols=['CIK', 'Form Type', 'Date Filed', 'Filename'],
                parse_dates=['Date Filed'],
            )
            print(listing)
            answer = input()
            if answer == 'Y':
                pathlib.Path(f"{DATA_DIR_LISTING}/{YEAR}QTR{QTR}_XBRL.gz").rename(
                    f"{DATA_DIR_LISTING}/{YEAR}QTR{QTR}_LIST.gz")
        except Exception as e:
            print(e)

--------------------------------------------------------------------------------
Processing 2010/1
       CIK   Form Type Date Filed  \
0    1000697    10-K   2010-02-26   
1    1001039    10-Q   2010-02-09   
2    1001082    10-K   2010-03-01   
3    1001838    10-K   2010-02-26   
4    1002638    10-Q   2010-02-04   
..       ...       ...        ...   
508    96223    10-K   2010-02-26   
509    97216    10-K   2010-02-24   
510    97476    10-K   2010-02-23   
511    97745    10-K   2010-02-26   
512     9892    10-K   2010-02-25   

                                         Filename                                      
0     https://sec.gov/Archives/edgar/data/1000697/000095012310017583/wat-20091231.xml  
1     https://sec.gov/Archives/edgar/data/1001039/000119312510025949/dis-20100102.xml  
2    https://sec.gov/Archives/edgar/data/1001082/000095012310018671/dish-20091231.xml  
3    https://sec.gov/Archives/edgar/data/1001838/000110465910010334/scco-20091231.xml  
4    https://sec

In [54]:

df = listing[listing['CIK'] == 8868]
df['Filename'] = df['Filename'].apply(lambda x: None)
df.loc[df['Filename'].isna()]

/tmp/ipykernel_51284/2532396748.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Filename'] = df['Filename'].apply(lambda x: None)


,CIK,Form Type,Date Filed,Filename
451,8868,10-K,2010-02-25,None


In [9]:
for index, row in listing.iterrows():
    # --------------------------------------------------------------------------------
    # Download XBRL XML
    # --------------------------------------------------------------------------------
    url = row['Filename']
    content = http_get_content(url, EDGAR_HTTP_HEADERS)

    # --------------------------------------------------------------------------------
    # Save
    # --------------------------------------------------------------------------------
    elements = url.split('/')
    filename = elements[-1]
    accession = elements[-2]
    cik = elements[-3]
    assert str(row['CIK']) == cik, f"CIK [{row['CIK']})] must match CIK part [{cik}] in url {url}"

    directory = f"{DATA_DIR_XBRL}{os.sep}{cik}{os.sep}{accession}"
    Logger.debug(f"worker(): saving XML to [{directory}:{filename}]...")

    save_to_xml(content, directory, filename)

DEBUG:sec_edgar_utility:http_get_content(): GET url [https://sec.gov/Archives/edgar/data/1000697/000095012310017583/wat-20091231.xml] headers [{'User-Agent': 'Company Name myname@company.com'}]
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sec.gov:443
DEBUG:urllib3.connectionpool:https://sec.gov:443 "GET /Archives/edgar/data/1000697/000095012310017583/wat-20091231.xml HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.sec.gov:443
DEBUG:urllib3.connectionpool:https://www.sec.gov:443 "GET /Archives/edgar/data/1000697/000095012310017583/wat-20091231.xml HTTP/1.1" 200 76282


worker(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1000697/000095012310017583:wat-20091231.xml]...


DEBUG:__main__:worker(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1000697/000095012310017583:wat-20091231.xml]...
DEBUG:sec_edgar_download_xbrl_xml:save_to_xml(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1000697/000095012310017583/wat-20091231.xml]...
DEBUG:sec_edgar_utility:http_get_content(): GET url [https://sec.gov/Archives/edgar/data/1001039/000119312510025949/dis-20100102.xml] headers [{'User-Agent': 'Company Name myname@company.com'}]
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sec.gov:443
DEBUG:urllib3.connectionpool:https://sec.gov:443 "GET /Archives/edgar/data/1001039/000119312510025949/dis-20100102.xml HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.sec.gov:443
DEBUG:urllib3.connectionpool:https://www.sec.gov:443 "GET /Archives/edgar/data/1001039/000119312510025949/dis-20100102.xml HTTP/1

worker(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1001039/000119312510025949:dis-20100102.xml]...


DEBUG:__main__:worker(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1001039/000119312510025949:dis-20100102.xml]...
DEBUG:sec_edgar_download_xbrl_xml:save_to_xml(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1001039/000119312510025949/dis-20100102.xml]...
DEBUG:sec_edgar_utility:http_get_content(): GET url [https://sec.gov/Archives/edgar/data/1001082/000095012310018671/dish-20091231.xml] headers [{'User-Agent': 'Company Name myname@company.com'}]
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sec.gov:443
DEBUG:urllib3.connectionpool:https://sec.gov:443 "GET /Archives/edgar/data/1001082/000095012310018671/dish-20091231.xml HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.sec.gov:443
DEBUG:urllib3.connectionpool:https://www.sec.gov:443 "GET /Archives/edgar/data/1001082/000095012310018671/dish-20091231.xml HTT

...

DEBUG:__main__:worker(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1013871/000095012310015824:nrg-20091231.xml]...
DEBUG:sec_edgar_download_xbrl_xml:save_to_xml(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1013871/000095012310015824/nrg-20091231.xml]...
DEBUG:sec_edgar_utility:http_get_content(): GET url [https://sec.gov/Archives/edgar/data/1014473/000119312510043162/vrsn-20091231.xml] headers [{'User-Agent': 'Company Name myname@company.com'}]
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sec.gov:443
DEBUG:urllib3.connectionpool:https://sec.gov:443 "GET /Archives/edgar/data/1014473/000119312510043162/vrsn-20091231.xml HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.sec.gov:443
DEBUG:urllib3.connectionpool:https://www.sec.gov:443 "GET /Archives/edgar/data/1014473/000119312510043162/vrsn-20091231.xml HTT

worker(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1014473/000119312510043162:vrsn-20091231.xml]...


DEBUG:__main__:worker(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1014473/000119312510043162:vrsn-20091231.xml]...
DEBUG:sec_edgar_download_xbrl_xml:save_to_xml(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1014473/000119312510043162/vrsn-20091231.xml]...
DEBUG:sec_edgar_utility:http_get_content(): GET url [https://sec.gov/Archives/edgar/data/1014739/000101473910000004/bios-20091231.xml] headers [{'User-Agent': 'Company Name myname@company.com'}]
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sec.gov:443
DEBUG:urllib3.connectionpool:https://sec.gov:443 "GET /Archives/edgar/data/1014739/000101473910000004/bios-20091231.xml HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.sec.gov:443
DEBUG:urllib3.connectionpool:https://www.sec.gov:443 "GET /Archives/edgar/data/1014739/000101473910000004/bios-20091231.xml H

worker(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1014739/000101473910000004:bios-20091231.xml]...


DEBUG:__main__:worker(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1014739/000101473910000004:bios-20091231.xml]...
DEBUG:sec_edgar_download_xbrl_xml:save_to_xml(): saving XML to [/home/oonisim/home/repository/git/oonisim/python_programs/finance/SEC/EDGAR/src/../data/XBRL/1014739/000101473910000004/bios-20091231.xml]...




---